In [10]:
import h5py
from pathlib import Path
import numpy as np

N = 2
path = Path(f'../data/raw/eventhdr/eval/{N}.h5')
COMBINE_FRAMES = 5


with h5py.File(path, "r") as file:
    p = np.array(file['events/ps'])
    t = np.array(file['events/ts'])
    x = np.array(file['events/xs'])
    y = np.array(file['events/ys'])
    frame_ts = []
    all_frame_ts = []
    print(file.attrs.keys())
    # print(file.attrs['duration'])
    print(file.attrs.get('source', 'unknown'))
    frames = []
    all_frames = []
    resolution = file.attrs.get('sensor_resolution')
    for i, img_name in enumerate(file['images']):
        all_frame_ts.append(file[f'images/{img_name}'].attrs['timestamp'])
        all_frames.append(np.array(file['images'][img_name]))
        if i % COMBINE_FRAMES != 0:
            continue
        frame_ts.append(file[f'images/{img_name}'].attrs['timestamp'])
        frames.append(np.array(file['images'][img_name]))

len(p)

<KeysViewHDF5 ['duration', 'num_events', 'num_flow', 'num_imgs', 'num_neg', 'num_pos', 'sensor_resolution', 't0', 'tk']>
unknown


3904717

In [3]:
frames_np = np.stack(all_frames) / (255**2)
frames_np = frames_np / frames_np.max()

In [21]:
import evs_explorer

from dynamic_fusion.data_generator.event_generator import ImageGenerator

EVS_EXPLORER_CONFIG = "../configs/data_generator/simulator/evs_explorer.yml"
DAVIS_CONFIG = "../configs/data_generator/simulator/davis_model.yml"
THRESHOLD = 1.3
MIN_ILLUMINANCE = 650
MAX_ILLUMINANCE = 12500

image_generator = ImageGenerator(data=frames_np*255, fps=500, num_frames=len(all_frames))


scfg = evs_explorer.Configuration.from_yaml(
    simulator_config=EVS_EXPLORER_CONFIG,
    sensor_config=DAVIS_CONFIG,
    sensor_model="davis_model",
)

scfg.input.source = image_generator
scfg.optics.max_illuminance_lux = MAX_ILLUMINANCE
scfg.optics.min_illuminance_lux = MIN_ILLUMINANCE
scfg.sensor.ONth_mul = THRESHOLD
scfg.sensor.OFFth_mul = THRESHOLD

evs_explorer = evs_explorer.EvsExplorer(scfg)
events = evs_explorer.run("sensor_data")
print(len(events))

Loading simulation config file from ../configs/data_generator/simulator/evs_explorer.yml
Loading sensor config file from ../configs/data_generator/simulator/davis_model.yml
GPU is available and will be used (cuda:0).
initializing pytorch / CUDA ... done
Running EVS Explorer 2.0.0
Loading frames from: ImageGenerator
The given update noise frequency will allow only to see up to 10.0 Hz of noise, but it actually goes up to 147 Hz on average!
dumping config to output/config/ImageGenerator/sim_config.yml
dumping config to output/config/ImageGenerator/sensor_config.yml


 65%|██████▌   | 261/401 [00:03<00:02, 51.29it/s]



 99%|█████████▉| 397/401 [00:05<00:00, 90.40it/s]

timing:
 - FramesExtractor: 0.00s = 0.00%
 - ConvertLuxToIrradiance: 0.00s = 0.06%
 - StdoutProgressBar: 0.03s = 0.50%
 - Convert8bitToLux: 0.04s = 0.79%
 - ConvertToTorch: 0.06s = 1.05%
 - SensorModelNode: 5.18s = 95.94%
All nodes: 5.31 seconds
Total runtime: 5.40 seconds
Total simulation time: 0.80 seconds
 -> simulation speed: 6.748633682727814 seconds realtime per second of simulation
output: sensor_data
499733


100%|██████████| 401/401 [00:23<00:00, 90.40it/s]

: 